# Load

In [53]:
import pandas as pd
from data_retrieval import connect_db, fetch_trx_data, fetch_cat_data, fetch_cat1_data, fetch_private_label_data, fetch_sustainability_data, fetch_segment_data
from data_processing import multi_aggregate_data, apply_custom_calculations
from data_analysis import  create_baskets_365_qty, flatten_baskets
from modelling import create_cooccurrence_matrix_with_recommendations, create_cooccurrence_matrix_with_recommendations_2
from modeling_2 import get_top_5_cat3_items, get_top_5_cat1_items, create_cat3_to_top_item_map
from modeling_3 import replace_item_cde_with_cat3_set,map_and_add_recommendations, map_and_add_recommendations_cat1, transform_recommendations, reorder_private
from modeling_4 import remove_duplicate_and_self_references, shift_recommendations_left, remove_empty_related_items, drop_spaces

In [54]:
# Sittun's code review edit suggestions

#import modelling as m1
#import modeling_2 as m2

In [55]:
#m2.replace_low_values()

In [56]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [57]:
# Load items 
df = pd.read_excel('data/AB List_dc_removed.xlsx', usecols=['Item Number'])
print(f">> Loaded {len(df)} e-commerce items with their item_cde")

>> Loaded 4291 e-commerce items with their item_cde


In [58]:
#Rename the columns
df.rename(columns={'Item Number': 'item_cde'}, 
          inplace=True)
print(f">> Renamed the columns to {df.columns.tolist()}")

>> Renamed the columns to ['item_cde']


## Fetch segment data

In [59]:
# connecting to db
conn = connect_db()
print(">> Connected to database")

>> Connected to database


In [60]:
# getting segment data from db
seg_df = fetch_segment_data(conn, df)
print(f">> Fetched the segment data containing {len(seg_df)} rows")

>> Fetched the segment data containing 4291 rows


In [61]:
#filter Packaging segment
seg_pkg_df=seg_df[seg_df['segment'].str.contains('Packaging', na=False)]

In [62]:
# connecting to db
conn = connect_db()
print(">> Connected to database")

>> Connected to database


In [63]:
# getting transaction data from db
trx_seg_df = fetch_trx_data(conn, seg_pkg_df)
print(f">> Fetched the transaction data containing {len(trx_seg_df)} rows")

>> Fetched the transaction data containing 279268 rows


## If Manufacturer filteration is needed (when scope not given)

In [64]:
#trx_df_new = trx_df[trx_df['mfg_name'].str.contains('3M', na=False)]


In [65]:
# OPTIONAL: Apply if mfg filter needed
#mfg_df=filter_mfg_name(trx_df, '')

In [66]:
# aggregate data at bill-to
aggregated_df = multi_aggregate_data(trx_seg_df)
print(f">> Aggregated data at bill-to level.Length: {len(aggregated_df)} rows")

>> Aggregated data at bill-to level.Length: 9608 rows


In [67]:
# custom column calculation for avg time interval
custom_calculated_df = apply_custom_calculations(aggregated_df)
print(f">> Calculated custom columns.Length: {len(custom_calculated_df)} rows")

>> Calculated custom columns.Length: 9608 rows


In [68]:
# make baskets for all possible starting date
baskets_df_365_qty = create_baskets_365_qty(custom_calculated_df)
print(f">> Made baskets with size: {len(baskets_df_365_qty)} baskets")

>> Made baskets with size: 9608 baskets


In [69]:
# flatten the baskets
flattened_basket_list_365 = flatten_baskets(baskets_df_365_qty)
print(f">> Flattened baskets. Length: {len(flattened_basket_list_365)}")

>> Flattened baskets. Length: 136111


In [70]:
# make item level recommendations
co_occurrence_matrix, recommendation_df = create_cooccurrence_matrix_with_recommendations_2(flattened_basket_list_365)
print(f">> Made item level recommendations for: {len(recommendation_df)} items")

100%|██████████| 1335/1335 [00:00<00:00, 1447.73it/s]


>> Made item level recommendations for: 1335 items


In [71]:
# Convert all values to integers, ignoring NaN
recommendation_df = recommendation_df.apply(pd.to_numeric, errors='coerce')

# Create the desired DataFrame
result_df = pd.DataFrame({
    'item_cde': recommendation_df.index,
    'reco': recommendation_df.apply(lambda row: [int(x) for x in row.dropna()], axis=1)
})



# Filter for sustainable

In [72]:
# connecting to db
conn = connect_db()
print(">> Connected to database")

>> Connected to database


In [73]:
# Get sustainable label information
sustainable_df = fetch_sustainability_data(conn, df)
print(f">> Fetched the sustainable label data containing {len(sustainable_df)} rows")

>> Fetched the sustainable label data containing 1833 rows


In [74]:
# Convert sustainable_df['item_cde'] to integers
sustainable_df['item_cde'] = sustainable_df['item_cde'].astype(int)

# Create a set of sustainable items
sustainable_set = set(sustainable_df['item_cde'])

# Filter reco list based on sustainable_df
result_df['reco'] = result_df['reco'].apply(lambda lst: [x for x in lst if x in sustainable_set])

In [75]:
# segment_sustainable items
df_pkg_sustainable = seg_pkg_df[seg_pkg_df['item_cde'].astype(int).isin(sustainable_set)].copy()


In [76]:
# Filter trx_df_pkg and keep 'item_cde' as a string
trx_df_sustainable = trx_seg_df[trx_seg_df['item_cde'].astype(int).isin(sustainable_set)].copy()

# Ensure 'item_cde' remains as string
trx_df_sustainable['item_cde'] = trx_df_sustainable['item_cde'].astype(str)
#trx_df_sustainable

In [77]:
result_df = result_df[result_df['reco'].apply(lambda x: len(x) > 0)]


In [78]:
# Get the cat3 top 5 items
top_5_items_cat3=get_top_5_cat3_items(trx_df_sustainable)
print(f">> Got cat3 top 5 items: {len(top_5_items_cat3)}")

>> Got cat3 top 5 items: 37


In [79]:
# Get the cat1 top 5 items
top_5_items_cat1=get_top_5_cat1_items(trx_df_sustainable)
print(f">> Got cat1 top 5 items: {len(top_5_items_cat1)}")

>> Got cat1 top 5 items: 10


In [80]:
conn = connect_db()
print(">> Connected to database")

>> Connected to database


In [81]:
# getting cat3 data from db
conn = connect_db()

cat3_df = fetch_cat_data(conn, df)
print(f">> Fetched the cat3 data containing {len(cat3_df)} rows")

%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 

In [82]:
conn = connect_db()
print(">> Connected to database")

>> Connected to database


In [83]:
# getting cat1 data from db
cat1_df = fetch_cat1_data(conn, df)
print(f">> Fetched the cat1 data containing {len(cat1_df)} rows")

%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 

In [84]:
# Map item to top item in cat3
item_to_cat3_top_map= create_cat3_to_top_item_map(df, cat3_df, top_5_items_cat3)
print(f">> Map items from item to top items in cat3 {len(item_to_cat3_top_map)} rows")

>> Map items from item to top items in cat3 1165 rows


# Cat 3 operations

In [85]:
# Make cat3 level baskets for co-occurance
cat3_basket_365=replace_item_cde_with_cat3_set(flattened_basket_list_365, cat3_df)
print(f">> Made cat3 level baskets for co-occurance size: {len(cat3_basket_365)}")

>> Made cat3 level baskets for co-occurance size: 136111


In [86]:
# Create co-occurance df for cat3 level
cat3_co_occurrence_matrix, cat3_recommendation_df=create_cooccurrence_matrix_with_recommendations_2(cat3_basket_365)
print(f">> Create co-occurance df for cat3 level df Length: {len(cat3_recommendation_df)}")

100%|██████████| 104/104 [00:00<00:00, 2938.00it/s]

>> Create co-occurance df for cat3 level df Length: 104


In [87]:
# Map cat3 and add cat3 level recos
new_cat3_recommendation_df = map_and_add_recommendations(seg_pkg_df, cat3_df, result_df, cat3_recommendation_df,top_5_items_cat3)
print(f">> Product+cat3 level df of Length: {len(new_cat3_recommendation_df)} rows")

>> Product+cat3 level df of Length: 1655 rows


In [88]:
# Drop empty space on new_cat3_recommendation_df

result_df_2=drop_spaces(new_cat3_recommendation_df)

In [89]:
result_df_2 = pd.DataFrame({
    'item_cde': new_cat3_recommendation_df.item_cde,
    'reco': new_cat3_recommendation_df.drop(columns=['item_cde'])  # Apply to all other columns
        .apply(lambda row: [
            int(x) if isinstance(x, (str, float, int)) and str(x).strip() != '' else x 
            for x in row.dropna()
        ], axis=1)
})

# Cat 1 Operations

In [90]:
# Make cat1 level baskets for co-occurance
cat1_basket_365=replace_item_cde_with_cat3_set(flattened_basket_list_365, cat1_df)
print(f">> Made cat1 level baskets for co-occurance size: {len(cat1_basket_365)}")

>> Made cat1 level baskets for co-occurance size: 136111


In [91]:
# Create co-occurance df for cat1 level
cat1_co_occurrence_matrix, cat1_recommendation_df=create_cooccurrence_matrix_with_recommendations(cat1_basket_365, top_n=5)
print(f">> Create co-occurance df for cat3 level df Length: {len(cat1_recommendation_df)}")

100%|██████████| 13/13 [00:00<?, ?it/s]

>> Create co-occurance df for cat3 level df Length: 13


In [92]:
# Map cat1 and add cat3 level recos
new_cat1_recommendation_df = map_and_add_recommendations_cat1(seg_pkg_df, cat1_df, new_cat3_recommendation_df, cat1_recommendation_df, top_5_items_cat1)
print(f">> Product+cat1+cat3 level df of Length: {len(new_cat1_recommendation_df)} rows")

>> Product+cat1+cat3 level df of Length: 1672 rows


In [93]:
result_df_3 = pd.DataFrame({
    'item_cde': new_cat1_recommendation_df.item_cde,
    'reco': new_cat1_recommendation_df.drop(columns=['item_cde'])  # Apply to all other columns
        .apply(lambda row: [
            int(x) if isinstance(x, (str, float, int)) and str(x).strip() != '' else x 
            for x in row.dropna()
        ], axis=1)
})

In [94]:
import pandas as pd
import itertools


# Ensure reco column contains lists and handle empty lists
result_df_3['reco'] = result_df_3['reco'].apply(lambda x: x if isinstance(x, list) else [])

# Flatten list of lists into a single list
result_df_3['reco'] = result_df_3['reco'].apply(lambda x: list(itertools.chain.from_iterable(x)) if any(isinstance(i, list) for i in x) else x)

# Sort each list and keep the first 5 min values
result_df_3['reco'] = result_df_3['reco'].apply(lambda x: x[:5])

# Convert the lists into separate columns
df_result_expanded = result_df_3.reco.apply(lambda x: pd.Series(x)).rename(columns=lambda i: f'Recommendation {i+1}')

# Merge with original DataFrame
df_3 = pd.concat([result_df_3.drop(columns=['reco']), df_result_expanded], axis=1)



In [95]:
df_3

,item_cde,Recommendation 1,Recommendation 2,Recommendation 3,Recommendation 4,Recommendation 5
0,10022692,10728033,10775537,10348094,10728306,10766745
1,10023733,11128976,10998558,10069016,11128965,10769359
2,10023749,10766745,10769395,10071824,10574994,10769391
3,10023970,10805476.0,10937514.0,10727930.0,NaN,NaN
4,10024232,10747826,10747828,11096322,10906194,10122093
...,...,...,...,...,...,...
1667,20042105,10824039,10727931,10769252,10553070,
1668,20050748,10728635,10727931,10442036,10553070,10348093
1669,20050754,10728635,10727931,10442036,10553070,10348093
1670,20050756,10728635,10727931,10442036,10553070,10348093


# Reorder private, alliance items

In [96]:
def reorder_alliance_new(df, pvt_a_label_df):
    """
    Reorders the recommendations for each item_cde based on private_label_sw
    and adds columns indicating whether each recommendation is private branded.
    Parameters:
        df: A DataFrame containing 'item_cde' and 'Recommendation 1' to 'Recommendation 15'.
        private_label_df: A DataFrame containing 'item_cde' and 'private_label_sw'.
    Returns:
        reordered_df: A DataFrame with reordered recommendations and private branding info.
    """
    pvt_a_label_df['Item Number'] = pvt_a_label_df['Item Number'].astype(str)

    alliance_brand_dict = {item: 1 if item in pvt_a_label_df['Item Number'].values else 0 for item in df['item_cde']}
    
    print(alliance_brand_dict)
    
    reordered_recommendations = []

    for _, row in df.iterrows():
        item_cde = row['item_cde']
        recommendations = [(row[f'Recommendation {i}'][0] if isinstance(row[f'Recommendation {i}'], list) else row[f'Recommendation {i}'],
        alliance_brand_dict.get(row[f'Recommendation {i}'][0] if isinstance(row[f'Recommendation {i}'], list) else row[f'Recommendation {i}']))
        for i in range(1, 6)]

        
        # Sort recommendations based on private_label_sw ('Y' should come first)
        recommendations.sort(key=lambda x: x[1] != 1)
        
        reordered_row = {'item_cde': item_cde}
        for i, (rec, private_label) in enumerate(recommendations):
            reordered_row[f'Recommendation {i+1}'] = rec
            reordered_row[f'Recommendation {i+1}_alliance'] = private_label
        
        reordered_recommendations.append(reordered_row)
    
    reordered_df = pd.DataFrame(reordered_recommendations)
    return reordered_df

In [97]:
alliance_df=pd.read_excel("data/Alliance + PB Scope - AB.xlsx")

# Re-order private items to top
reorder_private_df1 = reorder_alliance_new(new_cat1_recommendation_df, alliance_df)
print(f">> Re-ordered the private label data containing {len(reorder_private_df1)} rows")

{'10022692': 0, '10023733': 0, '10023749': 0, '10023970': 0, '10024232': 0, '10024244': 0, '10024260': 0, '10029049': 1, '10033257': 0, '10035501': 1, '10035502': 1, '10035503': 1, '10035504': 1, '10035614': 1, '10035634': 1, '10035734': 1, '10036186': 1, '10036223': 1, '10036268': 1, '10036270': 1, '10036272': 1, '10036273': 1, '10036277': 1, '10036547': 1, '10036549': 1, '10036550': 1, '10036553': 1, '10036584': 1, '10036645': 1, '10036699': 1, '10037717': 1, '10038031': 1, '10038032': 1, '10038049': 1, '10038106': 1, '10038134': 1, '10038135': 1, '10038136': 1, '10038137': 1, '10038836': 1, '10038874': 1, '10038908': 1, '10040875': 1, '10040884': 1, '10041051': 1, '10041098': 1, '10041298': 1, '10043468': 1, '10050643': 0, '10056561': 0, '10057742': 1, '10058192': 1, '10059275': 1, '10059312': 1, '10059431': 1, '10059950': 0, '10060537': 0, '10060599': 0, '10061066': 0, '10061675': 0, '10063950': 0, '10065002': 1, '10067401': 1, '10067455': 0, '10067458': 0, '10067650': 1, '10067997

In [98]:
conn = connect_db()
print(">> Connected to database")

>> Connected to database


In [99]:
# Get private label information
private_l_df = fetch_private_label_data(conn, df)
print(f">> Fetched the private label data containing {len(private_l_df)} rows")

>> Fetched the private label data containing 4291 rows


In [100]:
private_l_df['private_label_sw'].value_counts ()

private_label_sw
N    2581
Y    1709
Name: count, dtype: int64

In [101]:
def reorder_private_5(df, private_df):
    """
    Reorders the recommendations for each item_cde based on private_label_sw
    and adds columns indicating whether each recommendation is private branded.
    Parameters:
        df: A DataFrame containing 'item_cde' and 'Recommendation 1' to 'Recommendation 15'.
        private_label_df: A DataFrame containing 'item_cde' and 'private_label_sw'.
    Returns:
        reordered_df: A DataFrame with reordered recommendations and private branding info.
    """
    private_label_dict = dict(zip(private_df['item_cde'], private_df['private_label_sw']))
    
    reordered_recommendations = []

    for _, row in df.iterrows():
        item_cde = row['item_cde']
        recommendations = [(row[f'Recommendation {i}'], private_label_dict.get(row[f'Recommendation {i}'], 'N')) 
                           for i in range(1, 5)]
        
        # Sort recommendations based on private_label_sw ('Y' should come first)
        recommendations.sort(key=lambda x: x[1] != 'Y')
        
        reordered_row = {'item_cde': item_cde}
        for i, (rec, private_label) in enumerate(recommendations):
            reordered_row[f'Recommendation {i+1}'] = rec
            reordered_row[f'Recommendation {i+1}_private'] = private_label
        
        reordered_recommendations.append(reordered_row)
    
    reordered_df = pd.DataFrame(reordered_recommendations)
    return reordered_df

In [102]:
# Re-order private items to top
reorder_private_df2 = reorder_private_5(reorder_private_df1, private_l_df)
print(f">> Re-ordered the private label data containing {len(reorder_private_df2)} rows")

>> Re-ordered the private label data containing 1672 rows


# Sanity Check for all recommended items in sustainable

In [103]:
# Convert 'item_cde' column in items_pkg_sust_df to a set for fast lookup
sustainable_items_pkg_set = set(df_pkg_sustainable['item_cde'])

# Define a function to check if all recommended items exist in sustainable_items_set
def all_recos_in_sust(row):
    recommendations = [row[f'Recommendation {i}'] for i in range(1, 6) if pd.notna(row[f'Recommendation {i}'])]
    return all(item in sustainable_items_pkg_set for item in recommendations)

# Apply the check function to each row
reorder_private_df2['check'] = reorder_private_df2.apply(all_recos_in_sust, axis=1)



KeyError: 'Recommendation 5'

In [ ]:
# Remove duplicate recommendations and self-references
reorder_df_cleaned = remove_duplicate_and_self_references(df_3)
print(f">>Removed duplicate recommendations and self-references data containing {len(reorder_df_cleaned)} rows")

>>Removed duplicate recommendations and self-references data containing 1683 rows


In [ ]:
#Shift non empty recommendations to left
reorder_shift_left = shift_recommendations_left(reorder_df_cleaned)
print(f">>Shift non empty recommendations data containing {len(reorder_shift_left)} rows")

KeyError: 'Recommendation 6'

# Transform for enable

In [ ]:
# Transform the recommendations row-wise
transformed_df = transform_recommendations(reorder_shift_left)
print(f">> Transform the recommendations, new Length {len(transformed_df)} rows, which is 5 times {len(new_cat1_recommendation_df)} (Prev length)")

>> Transform the recommendations, new Length 8400 rows, which is 5 times 1680 (Prev length)


In [ ]:
# Remove empty items from related items column
nonempty_transformed_df=remove_empty_related_items(transformed_df)
print(f">>Data after removing empty related items have {len(nonempty_transformed_df)} rows")

>>Data after removing empty related items have 7902 rows


In [ ]:
nonempty_transformed_df

,Primary Item Number,Related Item Number
0,10012415,10098323
1,10012415,10546160
2,10012415,10943252
3,10012415,10097190
4,10012415,10049812
...,...,...
8393,20082513,10943252
8395,10370181,10169745
8397,10370181,20039761
8398,10370181,10943252


In [ ]:
#nonempty_transformed_df.to_csv('nonprint_minimum2_recommendations.csv', index=False)

## Result needed for new items only

In [ ]:
nonempty_transformed_df.to_csv('pkg_sustainable_dc.csv',index=False)